<a href="https://colab.research.google.com/github/ben854719/WASDE-Revenue-in-2024/blob/main/WASDE_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet langchain
!pip install --quiet langchain-google-genai
!pip install --quiet chromadb
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.4 MB/s eta 0:00:00


In [4]:
# import os
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass('GeminiAPI_Key')

GeminiAPI_Key··········


In [8]:
# import Bert as a useragent.

import os

os.environ['USER_AGENT'] = 'MyColabNotebook/1.0'

In [7]:
# import LangChain libraries
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma

In [9]:
# Import the dataset from the website of USDA (WASDE).
loader = WebBaseLoader("https://www.usda.gov/about-usda/general-information/staff-offices/office-chief-economist/commodity-markets/wasde-report")
docs = loader.load()

In [10]:
# Extract the text from the website of data content.
text_content = docs[0].page_content

# To select the required content.
text_content_0 = text_content.split("==References==")[0]
text_content_1 = text_content_0.split("==External Links==")[0]
text_content_2 = text_content_1.split("==See also==")[0]
final_text = text_content_2.split("==Notes==")[0]
print(final_text)

# Convert the text to LangChain's to the document format.
docs = [Document(page_content=final_text, metadata={"source":"usda"})]

























WASDE Report | Home






  Skip to main content



  Skip to main content











              An official website of the United States government
            
Here's how you know


Here's how you know


AskUSDA
News
Blog
Employee Resources









Official websites use .govA
                .gov Website belongs to an official government organization in the United States.
              






Secure .gov websites use HTTPS
A lock (
                
A Lock
Locked padlock icon

 ) or https:// means you’ve safely connected to the .gov website. Share sensitive information only on official, secure websites.
              












U.S. Department of Agriculture

Menu







U.S. Department of Agriculture















Home



Food






About FoodProviding a safety net for millions of Americans who are food-insecure and for developing and promoting dietary guidance based on scientific evidence.Discover How






Food Menu


Food Safety 


Food Safety Overvi

In [22]:
# Import Gemini chatbot.
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [23]:
# Save to disk.
VectorStore = Chroma.from_documents(
    documents=docs,
    embedding=gemini_embeddings,
    persist_directory="./chroma_db"
)

In [26]:
from re import search
# Load from disk.
VectorStore_disk = Chroma(
    persist_directory="./chroma_db",
    embedding_function=gemini_embeddings
)

# Use search_kwargs to set k=1
retriever = VectorStore_disk.as_retriever(search_kwargs={"k": 1})

# Get relevant documents and print the length
print(len(retriever.get_relevant_documents("MMLU")))

1


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", temparature=0.1, top_p=0.8, top_k=40)

In [28]:
from langchain_core.prompts import PromptTemplate

# Prompt template to query Gemini.
llm_prompt_template = """ You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible.\n
Question: {question} \nContext: {context} \nAnswer:"""

# Use keyword arguments to initialize PromptTemplate
llm_prompt = PromptTemplate(template=llm_prompt_template, input_variables=["question", "context"])

print(llm_prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} template=" You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum. Keep the answer as concise as possible.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [30]:
# Combine data from documents to readable string format.
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [33]:
# Prompt the model. I will show you to ask question to Gemini. Gemini will answer your question.
rag_chain.invoke("What is WASDE?")

'WASDE stands for World Agricultural Supply and Demand Estimates. It is a report that provides annual forecasts for supply and use of U.S. and world wheat, rice, coarse grains, oilseeds, and cotton.'

In [42]:
rag_chain.invoke("When WASDE releases Dates for 2025?")

'The WASDE report will release dates for 2025 on January 10, February 11, March 11, April 10, May 12, June 12, July 11, August 12, September 12, October 9, November 10, and December 9.'

In [47]:
rag_chain.invoke("Can you tell me about the press research?")

'I apologize, but I cannot answer your question as the provided text does not mention anything about press research.'